In [2]:
# Presales Entity Matching

import pandas as pd
import numpy as np
from rapidfuzz import fuzz
from unidecode import unidecode
from tqdm import tqdm

# === CONFIGURATION ===
INPUT_FILE = "presales_data_sample.csv"  # Path to the presales sample CSV
OUTPUT_FILE = "presales_match_results.csv"  # Where to save the matching results

# Thresholds
THRESHOLD_MATCH = 85       # Score >= this is Matched
THRESHOLD_REVIEW = 75      # Score >= this and < MATCH is Needs Review

# --- Normalize company names ---
def clean_name(name: str) -> str:
    """
    Normalize a company name by lowercasing, removing diacritics,
    stripping legal suffixes and punctuation, and collapsing whitespace.
    """
    if pd.isna(name):
        return ""
    s = unidecode(str(name)).lower().strip()
    legal_suffixes = [
        "inc", "inc.", "llc", "ltd", "gmbh", "ag", "srl",
        "sa", "bv", "plc", "co", "company", "corp",
        "corporation", "aps", "a/s"
    ]
    for suffix in legal_suffixes:
        if s.endswith(f" {suffix}"):
            s = s[: -len(suffix) - 1]
    for ch in ['-', ',', '.', '/', '"', "'", "(", ")"]:
        s = s.replace(ch, ' ')
    return ' '.join(s.split())

# --- Compute weighted fuzzy match score between input and candidate rows ---
def compute_match_score(input_row: pd.Series, candidate_row: pd.Series) -> float:
    name_input = clean_name(input_row['input_company_name'])
    candidate_name = clean_name(candidate_row['company_name'])
    candidate_legal = clean_name(candidate_row.get('company_legal_names', ''))

    # Fuzzy similarity on name vs legal
    score_name = fuzz.token_sort_ratio(name_input, candidate_name)
    score_legal = fuzz.token_sort_ratio(name_input, candidate_legal)
    best_name_score = max(score_name, score_legal)

    # Country and region bonuses
    bonus = 0.0
    if 'input_main_country' in input_row and 'main_country' in candidate_row:
        if str(input_row['input_main_country']).strip().lower() == str(candidate_row['main_country']).strip().lower():
            bonus += 0.25  # +25%
    if 'input_main_region' in input_row and 'main_region' in candidate_row:
        if str(input_row['input_main_region']).strip().lower() == str(candidate_row['main_region']).strip().lower():
            bonus += 0.15  # +15%

    final_score = best_name_score + bonus * 100
    return min(final_score, 100)

# --- Core matching function (returns detailed rows) ---
def presales_entity_matching(df: pd.DataFrame) -> pd.DataFrame:
    results = []

    for input_key, group in tqdm(df.groupby('input_row_key'), desc="Matching entities"):
        input_row = group.iloc[0]
        best_score = -1.0
        best_candidate = None

        # Evaluate each candidate
        for _, candidate_row in group.iterrows():
            score = compute_match_score(input_row, candidate_row)
            if score > best_score:
                best_score = score
                best_candidate = candidate_row

        # Determine match status
        if best_score >= THRESHOLD_MATCH:
            status = 'Matched'
            review = False
        elif best_score >= THRESHOLD_REVIEW:
            status = 'Needs Review'
            review = True
        else:
            status = 'Unmatched'
            review = False

        # Append one detailed result per input_key
        results.append({
            # Input fields
            'input_row_key': input_key,
            'input_company_name': input_row['input_company_name'],
            'input_main_country_code': input_row.get('input_main_country_code', None),
            'input_main_country': input_row.get('input_main_country', None),
            'input_main_region': input_row.get('input_main_region', None),
            'input_main_city': input_row.get('input_main_city', None),
            # Candidate fields
            'matched_veridion_id': best_candidate['veridion_id'] if best_candidate is not None else np.nan,
            'matched_company_name': best_candidate['company_name'] if best_candidate is not None else np.nan,
            'company_legal_names': best_candidate.get('company_legal_names', None) if best_candidate is not None else None,
            'main_country_code': best_candidate.get('main_country_code', None) if best_candidate is not None else None,
            'main_country': best_candidate.get('main_country', None) if best_candidate is not None else None,
            'main_region': best_candidate.get('main_region', None) if best_candidate is not None else None,
            'main_city': best_candidate.get('main_city', None) if best_candidate is not None else None,
            'main_postcode': best_candidate.get('main_postcode', None) if best_candidate is not None else None,
            'main_street': best_candidate.get('main_street', None) if best_candidate is not None else None,
            'main_street_number': best_candidate.get('main_street_number', None) if best_candidate is not None else None,
            'linkedin_url': best_candidate.get('linkedin_url', None) if best_candidate is not None else None,
            'website_url': best_candidate.get('website_url', None) if best_candidate is not None else None,
            # Meta
            'match_score': round(best_score, 1),
            'match_status': status,
            'review_needed': review
        })

    return pd.DataFrame(results)

# === Script entry point ===
if __name__ == '__main__':
    df = pd.read_csv(INPUT_FILE)
    results_df = presales_entity_matching(df)
    results_df.to_csv(OUTPUT_FILE, index=False)

    # Summary
    total = len(results_df)
    matched = (results_df['match_status'] == 'Matched').sum()
    needs_review = (results_df['match_status'] == 'Needs Review').sum()
    unmatched = (results_df['match_status'] == 'Unmatched').sum()

    print("✅ Matching completed.")
    print(f"Results saved to '{OUTPUT_FILE}'")
    print("Summary:")
    print(f"Total inputs: {total}")
    print(f"Matched: {matched}")
    print(f"Needs Review: {needs_review}")
    print(f"Unmatched: {unmatched}")
    print(f"% Matched: {matched/total*100:.1f}%")

    # --- Duplicate veridion_id check ---
    dupes = results_df[results_df.duplicated(subset=['matched_veridion_id'], keep=False) & results_df['matched_veridion_id'].notna()]
    dup_count = dupes['matched_veridion_id'].nunique()
    print(f"Duplicate veridion_id count: {dup_count}")
    if dup_count > 0:
        dupes.to_csv("presales_duplicate_veridion_ids.csv", index=False)
        print("Details of duplicates saved to 'presales_duplicate_veridion_ids.csv'.")


Matching entities: 100%|██████████| 592/592 [00:00<00:00, 1011.27it/s]

✅ Matching completed.
Results saved to 'presales_match_results.csv'
Summary:
Total inputs: 592
Matched: 525
Needs Review: 47
Unmatched: 20
% Matched: 88.7%
Duplicate veridion_id count: 7
Details of duplicates saved to 'presales_duplicate_veridion_ids.csv'.
